In [2]:
import pandas as pd
import numpy as np
from datetime import time, date, timedelta
import plotly.graph_objects as go

In [3]:
multi = pd.read_csv(r'C:\Users\Admin\Desktop\Fundamental Strategy\multi-factor\checkrobvre_return.csv')
cfo = pd.read_csv(r'C:\Users\Admin\Desktop\Fundamental Strategy\pcfo\checkpcfo_return.csv')
ps = pd.read_csv(r'C:\Users\Admin\Desktop\Fundamental Strategy\ptos\checkps_return05.csv')
volpri = pd.read_csv(r'C:\Users\Admin\Desktop\Foreigner_Strategy\check-vlpri-return.csv')
volpri.drop(columns = 'Unnamed: 0', inplace = True)
volpri = volpri.rename(columns = {'index':'date'})

In [4]:
for stock_df in [ps, cfo, multi]:
    stock_df['count'] = stock_df.iloc[:,1:len(stock_df.columns.to_list())].count(axis = 1) #exclude date column
    stock_df.loc[stock_df['count']<5,'profit'] = stock_df.iloc[:,1:len(stock_df.columns.to_list())-2].mean(axis = 1,skipna=True)*(stock_df['count']/5)
    stock_df.loc[stock_df['count']==0,'profit'] = 0
    stock_df.loc[stock_df['count']>=5,'profit'] = stock_df.iloc[:,1:len(stock_df.columns.to_list())-2].mean(axis = 1,skipna=True)
    stock_df['cumulative'] = (stock_df['profit'] +1).cumprod()

In [5]:
def annual_return(df):
    df['year'] = pd.to_datetime(df['date']).dt.year
    df.loc[df['year']!=df['year'].shift(1), 'choose']=1
    df1 = df[['year','cumulative','choose']]
    df1 = df1.dropna()
    df1['ann_return'] =df1['cumulative']/ df1['cumulative'].shift(1)-1
    df1['year'] = df1['year'].shift(1)
    df1['year'] = df1['year'].apply(lambda x:str(x)[0:4])
    df1 = df1[['year','ann_return']]
    df1 = df1.dropna()
    return df1
    

In [6]:
df2 = pd.DataFrame()
for df, name in zip([volpri, ps, cfo, multi],['volpri', 'ps', 'cfo', 'multi']):
    df1 = annual_return(df)
    df1 = df1.rename(columns = {'ann_return':name})
    if df2.empty:
        df2 = df1
    else:
        df2 = pd.merge(df2, df1, on =['year'], how = 'left')
    

In [7]:
df2 = df2.dropna()
df2 = df2.set_index('year')
df2['average_return'] = df2.mean(axis = 1,skipna=True)
df2 

,volpri,ps,cfo,multi,average_return
year,,,,,
2015,0.159415,0.335484,0.338600,2047.0,511.958375
2016,0.920836,1.007160,0.809802,4095.0,1024.434450
2017,1.995668,1.410005,1.053296,4095.0,1024.864742
2018,-0.154189,-0.173242,0.082952,4095.0,1023.688881
2019,0.169647,0.152438,-0.044442,4095.0,1023.819411
2020,0.654872,1.247258,2.587795,4095.0,1024.872481


In [ ]:
df2.dropcp;

In [9]:
df2.reset_index(inplace=True)
fig = go.Figure(data=[
#     go.Bar(x=df2['year'].tolist(), y=df2['average_return'],
#                             name='average_return'),
#     go.Bar(x=df2['year'].tolist(), y=df2['multi'],
#                             name='multi'),
    go.Bar(x=df2['year'].tolist(), y=df2['cfo'],
                            name='cfo'),
    go.Bar(x=df2['year'].tolist(), y=df2['ps'],
                            name='ps'),
    go.Bar(x=df2['year'].tolist(), y=df2['volpri'],
                            name='volpri')
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()